In [1]:
# LE BUT : avoir par dispositif DEPHY_FERME un seul et unique numéro DEPHY en DETAILLE, passer les autres en ALLEGE. Nettoyaer les code_dephy qui sont pas bons et trouver les numéros DEPHY manquants !

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

In [3]:
path = '/home/administrateur/Bureau/Datagrosyst/data_entrepot_outils/'
donnees = dict(
    sdc = pd.read_csv(path + 'sdc.csv'),
    dispositif = pd.read_csv(path + 'dispositif.csv'),
    domaine = pd.read_csv(path + 'domaine.csv'),
    intv_real = pd.read_csv(path + 'intervention_realise_agrege.csv', usecols=['id','sdc_id']),
    intv_synth = pd.read_csv(path + 'intervention_synthetise_agrege.csv', usecols=['id','sdc_id'])
)

In [4]:
sdc = donnees['sdc'][['id','code','nom','code_dephy','reseaux_ir','dispositif_id','modalite_suivi_dephy']].rename(columns={
    'id':'sdc_id',
    'code':'sdc_code',
    'nom':'sdc_nom'})
dispositif = donnees['dispositif'][['id','code','type','domaine_id']].rename(columns={
    'id':'dispositif_id',
    'code':'dispositif_code',
    'type':'dispositif_type'})
domaine = donnees['domaine'][['id','nom','code','commune_id','campagne']].rename(columns={
    'id':'domaine_id',
    'code':'domaine_code',
    'nom' : 'domaine_nom'})
intv = pd.concat([donnees['intv_real'], donnees['intv_synth']], ignore_index=True)

df = sdc.merge(dispositif, on='dispositif_id', how='left')
df = df.merge(domaine, on='domaine_id', how='left')

In [5]:
dfferme = df.loc[(df['dispositif_type']=='DEPHY_FERME')].drop(columns=['dispositif_type'])

In [6]:
pattern_exact = r'^[A-Z]{3}(?:[0-9]|X)\d{4}$'
pattern_approx = r'([A-Z]{3}(?:[0-9]|X)\d{4})(?!\d)'
pattern_bizarre = r'^[A-Z]{3}\d{4}$'

def transform_value(x):
    if len(x) == 0:
        return None
    elif len(x) == 1:
        return x[0]
    else:
        return x 

In [7]:
mask_vide = (dfferme['code_dephy'].isnull())
mask_allege = (dfferme['modalite_suivi_dephy']=='ALLEGE')
mask_pattern_exact = dfferme['code_dephy'].str.fullmatch(pattern_exact, na=False)
mask_pattern_approx = dfferme['code_dephy'].str.fullmatch(pattern_approx, na=False)

print(
    "Nombre de numéro DEPHY\n",
    "     saisis :",dfferme.loc[~mask_vide].shape[0],"\n",
    "     vides :",dfferme.loc[mask_vide].shape[0],"\n",
    "     allégés mais saisis :",dfferme.loc[mask_allege & ~mask_vide].shape[0],"\n",
    "     allégés et vide :",dfferme.loc[mask_allege & mask_vide].shape[0],"\n",
    "     suivant le pattern :",dfferme.loc[mask_pattern_exact & ~mask_vide].shape[0],"\n",
    "     ne suivant pas le pattern mais saisis :",dfferme.loc[~mask_pattern_exact & ~mask_vide].shape[0]
)

Nombre de numéro DEPHY
      saisis : 29457 
      vides : 3043 
      allégés mais saisis : 0 
      allégés et vide : 1950 
      suivant le pattern : 28731 
      ne suivant pas le pattern mais saisis : 726


In [8]:
# 1. Nettoyage des numéros DEPHY
dfferme['code_dephy'] = dfferme['code_dephy'].astype(str).\
    str.replace('PPZ', '').\
        str.replace('_', '').\
            str.replace('-', '').\
                str.replace(' ', '').\
                    str.replace('\t', '').\
                        str.upper().\
    where(dfferme['code_dephy'].notna(), None)

dfferme['code_dephy'] = np.where(dfferme['code_dephy']=='', None, dfferme['code_dephy'])

In [9]:
mask_vide = ((dfferme['code_dephy'].isnull()))
mask_allege = (dfferme['modalite_suivi_dephy']=='ALLEGE')
mask_pattern_exact = dfferme['code_dephy'].str.fullmatch(pattern_exact, na=False)
mask_pattern_approx = dfferme['code_dephy'].str.fullmatch(pattern_approx, na=False)

print(
    "Nombre de numéro DEPHY, après nettoyage\n",
    "     saisis :",dfferme.loc[~mask_vide].shape[0],"\n",
    "     vides :",dfferme.loc[mask_vide].shape[0],"\n",
    "     allégés mais saisis :",dfferme.loc[mask_allege & ~mask_vide].shape[0],"\n",
    "     allégés et vide :",dfferme.loc[mask_allege & mask_vide].shape[0],"\n",
    "     suivant le pattern :",dfferme.loc[mask_pattern_exact & ~mask_vide].shape[0],"\n",
    "     ne suivant pas le pattern mais saisis :",dfferme.loc[~mask_pattern_exact & ~mask_vide].shape[0]
)

Nombre de numéro DEPHY, après nettoyage
      saisis : 29434 
      vides : 3066 
      allégés mais saisis : 0 
      allégés et vide : 1950 
      suivant le pattern : 29131 
      ne suivant pas le pattern mais saisis : 303


In [10]:
dfferme['tokeep'] = np.where(
    mask_allege & mask_vide, 'allege et vide',
    np.where(mask_pattern_exact, 'pattern valide',
        np.where(~mask_pattern_exact & ~mask_vide, 'pattern invalide',
            np.where(mask_vide, 'vide',
                None
))))

In [11]:
# 2. Mise à jour de la modalite_suivi_dephy en ALLEGE si le code DEPHY est manquant/invalide et que le dispositif_id contient d'autre sdc dont au moins un avec un code DEPHY valide

# On regarde tout quand même pour voir s'il y a des cas ou plusieurs sdc suivi détaillé en DEPHY sont dans le même dispositif !

dispo_cdephy = dfferme.groupby('dispositif_id').apply(lambda x: pd.Series({
    'code_dephy': list(x.loc[~x['code_dephy'].isnull() & x['code_dephy'].str.fullmatch(pattern_exact, na=False),'code_dephy'].unique()),
    'is_a_cdephy_nb': len(x.loc[x['code_dephy'].str.fullmatch(pattern_exact, na=False)]),
    'not_a_cdephy_nb': len(x.loc[~(x['code_dephy'].str.fullmatch(pattern_exact, na=False))])
    }),include_groups=False).reset_index()
dispo_cdephy['code_dephy'] = dispo_cdephy['code_dephy'].apply(transform_value)

sdc_et_dispo = dfferme.merge(dispo_cdephy.rename(columns={'code_dephy':'dispo_code_dephy'}), 
                         on='dispositif_id', how='left')

In [12]:
# 2.1. Probleme il y a des dispositifs qui réutilisent des numéros DEPHY valide sur plusieurs sdc !

sdc_cdephy_for_who = sdc_et_dispo.loc[
    (sdc_et_dispo['is_a_cdephy_nb'] >
    sdc_et_dispo['dispo_code_dephy'].apply(lambda x: len(x) if isinstance(x, list) else 1)) &
    (sdc_et_dispo['modalite_suivi_dephy'] != 'ALLEGE') &
    (sdc_et_dispo['code_dephy'].notnull())
]

In [13]:
interv_who_sdc_lvl = intv.merge(sdc_cdephy_for_who, on='sdc_id', how='right')
interv_who_sdc_lvl = interv_who_sdc_lvl.groupby('sdc_id').agg(
    {
        **{c: 'nunique' for c in ['id']},
        **{c: 'first' for c in sdc_cdephy_for_who.columns.difference(['id','sdc_id'])}
    }, include_groups=False).reset_index()
interv_who_sdc_lvl = interv_who_sdc_lvl.loc[interv_who_sdc_lvl['code_dephy'].str.fullmatch(pattern_exact, na=False)]

def get_unique_sdc_id_by_nb_intv_sup_0(x):
    mask_id = x['id'] > 0
    id_matches = x[mask_id]
    if len(id_matches) == 1:
        return id_matches['sdc_id'].iloc[0]
    return None

who_cdephy_dispo_lvl = interv_who_sdc_lvl.groupby(['dispositif_id', 'code_dephy']).apply(
    lambda x: pd.Series({
        'sdc_id_by_nb_intv_sup_0': get_unique_sdc_id_by_nb_intv_sup_0(x)
    }), include_groups=False
).reset_index()

In [14]:
sdc_multiple_cdephy_keep_and_suppr = sdc_cdephy_for_who.merge(who_cdephy_dispo_lvl, on=['dispositif_id','code_dephy'], how='left')

sdc_multiple_cdephy_keep_and_suppr['tokeepV1'] = np.where(
    sdc_multiple_cdephy_keep_and_suppr['sdc_id_by_nb_intv_sup_0'].isnull(),
    'besoin plsrs numero dephy >> NEED ATTENTION', 
        np.where(
        sdc_multiple_cdephy_keep_and_suppr['sdc_id_by_nb_intv_sup_0'] == sdc_multiple_cdephy_keep_and_suppr['sdc_id'],
        'unique sdc valide avec interv dans dispositif_id >> KEEP', 
        'code dephy dupliqué dans dispositif_id face à sdc ok >> SUPPR')
)

display(sdc_multiple_cdephy_keep_and_suppr)
print(sdc_multiple_cdephy_keep_and_suppr['tokeepV1'].value_counts())

dfferme=dfferme.merge(sdc_multiple_cdephy_keep_and_suppr[['sdc_id','tokeepV1']], on='sdc_id', how = 'left')

,sdc_id,sdc_code,sdc_nom,code_dephy,reseaux_ir,dispositif_id,modalite_suivi_dephy,dispositif_code,domaine_id,domaine_nom,domaine_code,commune_id,campagne,tokeep,dispo_code_dephy,is_a_cdephy_nb,not_a_cdephy_nb,sdc_id_by_nb_intv_sup_0,tokeepV1
0,fr.inra.agrosyst.api.entities.GrowingSystem_90...,c0951bc1-bc2f-4e32-b5da-4bf0a0e235d6,SDC cépages moyennement sensibles Marmorières,VIF31865,IR_BENNAMANE,fr.inra.agrosyst.api.entities.GrowingPlan_f9c6...,DETAILLE,db5c34e5-f0c7-45bf-a6d8-cd844af03754,fr.inra.agrosyst.api.entities.Domain_ee03198f-...,SCEA Marmorières,1441626e-1810-493c-bdb6-8bf44bcb180c,fr.inra.agrosyst.api.entities.referential.RefL...,2022,pattern valide,VIF31865,4,0,None,besoin plsrs numero dephy >> NEED ATTENTION
1,fr.inra.agrosyst.api.entities.GrowingSystem_f1...,223e862a-3be6-4b9c-aedd-95d67260c859,DEPHY EARL DANTEC,GCF38053,IR_LE DU,fr.inra.agrosyst.api.entities.GrowingPlan_b3b9...,DETAILLE,b4f0b233-6ee0-464b-bede-1305c8a1f2bb,fr.inra.agrosyst.api.entities.Domain_b1a503e5-...,EARL DANTEC,a27bed9c-3422-4d67-b88e-0dcdacd11db3,fr.inra.agrosyst.api.entities.referential.RefL...,2022,pattern valide,GCF38053,2,0,fr.inra.agrosyst.api.entities.GrowingSystem_f1...,unique sdc valide avec interv dans dispositif_...
2,fr.inra.agrosyst.api.entities.GrowingSystem_45...,8bdfa260-10cc-43c2-8138-4847d7d9113e,PPZ_PYF10363,PYF10363,IR_CANNO,fr.inra.agrosyst.api.entities.GrowingPlan_28d0...,DETAILLE,72be521c-a7c2-4ed0-8fb5-58c6339022ca,fr.inra.agrosyst.api.entities.Domain_1fe6467b-...,DOMAINE_GICQUEL,bbe34981-fcb7-4a07-b6e9-bfc8a95e7668,fr.inra.agrosyst.api.entities.referential.RefL...,2018,pattern valide,PYF10363,5,0,fr.inra.agrosyst.api.entities.GrowingSystem_45...,unique sdc valide avec interv dans dispositif_...
3,fr.inra.agrosyst.api.entities.GrowingSystem_3b...,6804ac5e-f56a-4e01-a447-85dfd6c319b7,SDC2 Herbe,PYF10486,IR_CANNO,fr.inra.agrosyst.api.entities.GrowingPlan_c2a6...,DETAILLE,cb1978f6-e9ca-4ead-8742-7863d1fa0039,fr.inra.agrosyst.api.entities.Domain_efcd5810-...,DOMAINE_BENOIT JEAN-MICHEL,a2800131-6246-4a46-89bd-5a159c335ada,fr.inra.agrosyst.api.entities.referential.RefL...,2014,pattern valide,PYF10486,2,0,fr.inra.agrosyst.api.entities.GrowingSystem_a8...,code dephy dupliqué dans dispositif_id face à ...
4,fr.inra.agrosyst.api.entities.GrowingSystem_3e...,d6f3c096-69ce-4efd-8257-703e5f853538,SDC 2 avec pature,GCF32104,IR_CANNO,fr.inra.agrosyst.api.entities.GrowingPlan_7104...,DETAILLE,1dc5449d-3639-4249-80cd-05d34419236e,fr.inra.agrosyst.api.entities.Domain_a5de21e5-...,Domaine NABUCET,aff1aaa9-0d68-4af7-aca5-a949f58cf640,fr.inra.agrosyst.api.entities.referential.RefL...,2020,pattern valide,GCF32104,2,0,fr.inra.agrosyst.api.entities.GrowingSystem_6f...,code dephy dupliqué dans dispositif_id face à ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,fr.inra.agrosyst.api.entities.GrowingSystem_ab...,8bf4b30e-263d-4674-bec7-f3974fd15e54,GCF33714_CBO,GCF33714,IR_MANGIN,fr.inra.agrosyst.api.entities.GrowingPlan_3d24...,DETAILLE,9064dd9b-610d-4077-8fbc-217f588f8fa5,fr.inra.agrosyst.api.entities.Domain_9132237d-...,DOMAINE_GAEC DU VIEUX NOYER,bd6f6e34-bac5-42e7-8c33-dca2d473ac0c,fr.inra.agrosyst.api.entities.referential.RefL...,2021,pattern valide,GCF33714,2,0,fr.inra.agrosyst.api.entities.GrowingSystem_ab...,unique sdc valide avec interv dans dispositif_...
1484,fr.inra.agrosyst.api.entities.GrowingSystem_6a...,3a773e93-3046-46ff-af32-9475eedc7635,PPZ_PYF10744,PYF10744,IR_PEILLERON,fr.inra.agrosyst.api.entities.GrowingPlan_b4e5...,DETAILLE,4023178c-7b67-4959-b3d5-b222647f6392,fr.inra.agrosyst.api.entities.Domain_71de3108-...,DOMAINE_CHAUSY,9b17da22-7163-4595-9ed9-8b24e050e083,fr.inra.agrosyst.api.entities.referential.RefL...,2024,pattern valide,PYF10744,3,0,fr.inra.agrosyst.api.entities.GrowingSystem_72...,code dephy dupliqué dans dispositif_id face à ...
1485,fr.inra.agrosyst.api.entities.GrowingSystem_8d...,3c048e58-5411-475c-a67c-be73db46e534,ROTATION RGA-MAIS-BLE;,PYF27726,NaN,fr.inra.agro

tokeepV1
code dephy dupliqué dans dispositif_id face à sdc ok >> SUPPR    628
unique sdc valide avec interv dans dispositif_id >> KEEP         482
besoin plsrs numero dephy >> NEED ATTENTION                      378
Name: count, dtype: int64


In [15]:
# 2.2. Voici la liste des code DEPHY qui sont en DETAILLE mais qui ne suivent pas le pattern attendu alors que d'autre sdc du même dispositif ont un code DEPHY valide

sdc_to_allege = sdc_et_dispo.loc[
    (sdc_et_dispo['modalite_suivi_dephy'] != 'ALLEGE') &
    (sdc_et_dispo['dispo_code_dephy'].notnull()) &
    ~(sdc_et_dispo['code_dephy'].str.fullmatch(pattern_exact, na=False))
    ]

In [16]:
dfferme.loc[ (dfferme['sdc_id'].isin(sdc_to_allege['sdc_id'])) & 
            (dfferme['code_dephy'].isnull()) ,
            'tokeepV2'] = 'detaillé + vide MAIS dispositif_id avec valide >> ALLEGE'

dfferme.loc[ (dfferme['sdc_id'].isin(sdc_to_allege['sdc_id'])) & 
            (~dfferme['code_dephy'].isnull()) ,
            'tokeepV2'] = 'detaillé + invalide MAIS dispositif_id avec valide >> ALLEGE'

dfferme.loc[ (~dfferme['sdc_id'].isin(sdc_to_allege['sdc_id'])) & 
            (~dfferme['code_dephy'].str.fullmatch(pattern_exact, na=False)) & 
            (dfferme['modalite_suivi_dephy'] != 'ALLEGE') , 
            'tokeepV2'] = 'detaillé + invalide|vide ET dispositif_id sans valide >> SEE TOKEEPV3'

print(
    "Nombre de sdc sans code DEPHY à passer en ALLEGE car dispositif avec au moins un code DEPHY valide :",
    len( dfferme.loc[ (dfferme['sdc_id'].isin(sdc_to_allege['sdc_id'])) & dfferme['code_dephy'].isnull() ] ),

    "\nNombre de sdc avec code DEPHY invalide à passer en ALLEGE car dispositif avec au moins un code DEPHY valide :",
    len( dfferme.loc[ (dfferme['sdc_id'].isin(sdc_to_allege['sdc_id'])) & ~dfferme['code_dephy'].str.fullmatch(pattern_exact, na=True) ] ),

    "\nNombre de sdc sans code DEPHY (invalide ou null) et sans dispositif avec au moins un code DEPHY valide :",
    len( dfferme.loc[ ~(dfferme['sdc_id'].isin(sdc_to_allege['sdc_id'])) & (~dfferme['code_dephy'].str.fullmatch(pattern_exact, na=False)) & (dfferme['modalite_suivi_dephy'] != 'ALLEGE') ] )
)

Nombre de sdc sans code DEPHY à passer en ALLEGE car dispositif avec au moins un code DEPHY valide : 962 
Nombre de sdc avec code DEPHY invalide à passer en ALLEGE car dispositif avec au moins un code DEPHY valide : 183 
Nombre de sdc sans code DEPHY (invalide ou null) et sans dispositif avec au moins un code DEPHY valide : 274


In [17]:
# 3. Pour ceux qui restent : les sdc qu'on arrive pas à tagué en ALLEGE, ni à nettoyer, on va essayer de trouver un numero dephy qui match via le nom ou les autres numéro DEPHY du meme sdc_code

sdc_restant = dfferme.loc[~(dfferme['sdc_id'].isin(sdc_to_allege['sdc_id'])) & 
            (~(dfferme['code_dephy'].str.fullmatch(pattern_exact, na=True)) | 
            (dfferme['code_dephy'].isnull())) & 
            (dfferme['modalite_suivi_dephy'] != 'ALLEGE') ]


sdc_bis = dfferme.groupby('sdc_code').apply(lambda x: pd.Series({
    'code_dephy_parSdcCode': list(set( [y for y in x['code_dephy'] if y is not np.nan] )),
    'code_dephy_parSdcCode_pattern': list(set( [y for y in x['code_dephy'].str.extract(pattern_approx)[0] if y is not np.nan] ))
}), include_groups=False).reset_index()
sdc_bis['code_dephy_parSdcCode'] = sdc_bis['code_dephy_parSdcCode'].apply(transform_value)
sdc_bis['code_dephy_parSdcCode_pattern'] = sdc_bis['code_dephy_parSdcCode_pattern'].apply(transform_value)

sdc_restant['code_dephy_parSdcNom_pattern'] = sdc_restant['sdc_nom'].str.extract(pattern_approx)

sdc_restant = sdc_restant.merge(sdc_bis, on='sdc_code', how='left')

In [18]:
numero_dephy_ajout_X = sdc_restant.loc[sdc_restant['code_dephy'].str.fullmatch(pattern_bizarre, na=False) & 
                sdc_restant['code_dephy_parSdcCode_pattern'].isnull() & 
                sdc_restant['code_dephy_parSdcNom_pattern'].isnull()]

numero_dephy_ajout_X_ou_non = sdc_restant.loc[sdc_restant['code_dephy'].str.fullmatch(pattern_bizarre, na=False) & 
                (sdc_restant['code_dephy_parSdcCode_pattern'].notnull() |
                sdc_restant['code_dephy_parSdcNom_pattern'].notnull())]

numero_dephy_find_otherway = sdc_restant.loc[~sdc_restant['code_dephy'].str.fullmatch(pattern_bizarre, na=False) & 
                (sdc_restant['code_dephy_parSdcCode_pattern'].notnull() |
                sdc_restant['code_dephy_parSdcNom_pattern'].notnull())]

aucune_idee = sdc_restant.loc[~sdc_restant['code_dephy'].str.fullmatch(pattern_bizarre, na=False) & 
                sdc_restant['code_dephy_parSdcCode_pattern'].isnull() &
                sdc_restant['code_dephy_parSdcNom_pattern'].isnull()]

print(
    "Nombre de sdc detaillé avec un code DEPHY bizarre à corriger en ajoutant un 'X' :",
    numero_dephy_ajout_X.shape[0],
    "\nNombre de sdc detaillé avec un code DEPHY bizarre et un code DEPHY trouvé via le nom ou les autres numéro DEPHY du même sdc_code :",
    numero_dephy_ajout_X_ou_non.shape[0],
    "\nNombre de sdc detaillé avec un code DEPHY trouvé via le nom ou les autres numéro DEPHY du même sdc_code :",
    numero_dephy_find_otherway.shape[0],
    "\nCeux là aucne idée de comment faire :",
    aucune_idee.shape[0]
)

numero_dephy_ajout_X['tokeepV3'] = 'pattern bizarre >> AJOUT X'
numero_dephy_ajout_X_ou_non['tokeepV3'] = 'pattern bizarre >> NEED ATTENTION'
numero_dephy_find_otherway['tokeepV3'] = 'pattern dans sdc_nom|sdc_code >> COORECTION'
aucune_idee['tokeepV3'] = 'introuvable ? >> NEED ATTENTION'

cols = ['sdc_id','tokeepV3','code_dephy_parSdcNom_pattern','code_dephy_parSdcCode_pattern']
sdc_reste = pd.concat([numero_dephy_ajout_X[cols], numero_dephy_ajout_X_ou_non[cols], numero_dephy_find_otherway[cols], aucune_idee[cols]])

dfferme = dfferme.merge(sdc_reste, on='sdc_id', how = 'outer')

Nombre de sdc detaillé avec un code DEPHY bizarre à corriger en ajoutant un 'X' : 25 
Nombre de sdc detaillé avec un code DEPHY bizarre et un code DEPHY trouvé via le nom ou les autres numéro DEPHY du même sdc_code : 6 
Nombre de sdc detaillé avec un code DEPHY trouvé via le nom ou les autres numéro DEPHY du même sdc_code : 48 
Ceux là aucne idée de comment faire : 195


In [19]:
dfferme.to_csv('/home/administrateur/Bureau/Datagrosyst/catalogue_script_agrosyst/02_outils/scripts/nettoyage_numdephy_resultat.csv', index=False)